# ⚾Group2
####   - DSI310 final assignment: Data Catalog Google Cloud Storage (GCS)
* 6424650031 แพรวา ประสาทไทย
* 6424650395 ศิวกร นิตย์กิจสมบูรณ์
* 6424650494 อธิชา เจริญธนกิจกุล
* 6424650551 เบญฤญา คำคงศักดิ์
* 6424650627 วิชญาพร ธนสิทธิโชค


In [2]:
import pandas as pd
import fsspec
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import os

def tree(fs, path, prefix=""):
    """Recursively list the contents of a directory in a tree-like format."""
    # print(path)
    items = fs.ls(path, detail=True)
    for i, item in enumerate(items):
        is_last = i == (len(items) - 1)  # Check if this is the last item
        if item['type'] == 'directory':
            # Print the directory name
            print(f"{prefix}{'└── ' if is_last else '├── '}{item['name'].split('/')[-1]}")
            # Recursively list this directory's contents
            new_prefix = prefix + ('    ' if is_last else '│   ')
            tree(fs, item['name'], new_prefix)
        else:
            # Print the file name
            print(f"{prefix}{'└── ' if is_last else '├── '}{item['name'].split('/')[-1]}")


### Local File System

In [8]:
# fs = fsspec.filesystem('file',auto_mkdir=True) #fsspec.filesystem(catalog_path, auto_mkdir=False)

# catalog_path='./catalog/'
# with fs.open(catalog_path+'readme.md','wb') as f:
#     f.write(b'# Hello')
    
# tree(fs,catalog_path)

### Google Cloud Storage(GCS)

In [3]:
gcs_token_path = r'teacher-dsi310-2023.json'
catalog_path = 'gcs://dsi310_bucket/group02'
fs=fsspec.filesystem('gcs', token= gcs_token_path)
tree(fs,catalog_path)


├── 2023-12-14
│   ├── 2022_MLB_Batting.parquet
│   ├── 2022_MLB_Pitching.parquet
│   ├── Batter_stats.parquet
│   ├── Pitcher_stats.parquet
│   └── Prospect.parquet
└── 


## Data set1

In [63]:
MLB_Batting_stats_df = pd.read_csv(r'C:\Users\ernie\Desktop\314 file\2022_MLB\Batting.csv',encoding='latin-1')


#Metadata
metadata = {
 'data_type': '2 = ข้อมูลสถิติ',
 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)ในปี 2022',
 'owner_org': 'Sports-Reference',
 'maintainer': ' Baseball-Reference',
 'maintainer_email': 'sf@sports-reference.com',
 'tag_string': 'Rk,Name,TM,TB',
 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น',
 'update_frequency_unit': 'X = ไม่มีการปรับเปลี่ยนหลังจากปี 2022',
 'update_frequency_interval': '1',
 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป',
 'data_source':'Baseball Refernce',
 'data_format':'99 = Parquet',
 'data_category':'1 = ข้อมูลสาธารณะ',
 'license_id':'1=Creative Commons Attributions'
}
data_dictionary = {
    'Rk': 'Rank',
    'Name': 'Player name',
    'Age': "Player's age",
    'Tm': 'Team',
    'Lg': 'League',
    'G': 'Games played',
    'PA': 'Plate appearances',
    'AB': 'At bats',
    'R': 'Runs scored/allowed',
    'H': 'Hits/hits allowed',
    '2B': 'Doubles hit/allowed',
    '3B': 'Triples hit/allowed',
    'HR': 'Home runs hit/allowed',
    'RBI': 'Runs batted in',
    'SB': 'Stolen bases',
    'CS': 'Caught stealing',
    'BB': 'Bases on balls/walks',
    'SO': 'Strikeouts',
    'BA': 'Hits/at bats',
    'OBP': '(H + BB + HBP) / (AB + BB + HBP + SF)',
    'SLG': 'Total bases/at bats or (1B + 2 * 2B + 3 * 3B + 4 * HR) / AB',
    'OPS': 'On-base + Slugging percentages',
    'OPS+': '100 * (OBP / logOBP + SLG / logSLG - 1)',
    'TB': 'Total bases',
    'GDP': 'Double plays grounded into',
    'HBP': 'Times hit by a pitch',
    'SH': 'Sacrifice hits',
    'SF': 'Sacrifice flies',
    'IBB': 'Intentional bases on balls'
}


# Convert DataFrame to PyArrow Table with metadata
table3 = pa.Table.from_pandas(MLB_Batting_stats_df)
table3 = table3.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table3.schema

Rk;Name;Age;Tm;Lg;G;PA;AB;R;H;2B;3B;HR;RBI;SB;CS;BB;SO;BA;OBP;SLG;OPS;OPS+;TB;GDP;HBP;SH;SF;IBB: string
-- schema metadata --
metadata: '{'data_type': '2 = ข้อมูลสถิติ', 'title'' + 1261
dictionary: '{'Rk': 'Rank', 'Name': 'Player name', 'Age': "Player's age",' + 767

In [67]:
# Extracting metadata and data dictionary
metadata = eval(table3.schema.metadata[b'metadata']) if b'metadata' in table3.schema.metadata else None
data_dictionary = eval(table3.schema.metadata[b'dictionary']) if b'dictionary' in table3.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': '2 = ข้อมูลสถิติ', 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)ในปี 2022', 'owner_org': 'Sports-Reference', 'maintainer': ' Baseball-Reference', 'maintainer_email': 'sf@sports-reference.com', 'tag_string': 'Rk,Name,TM,TB', 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น', 'update_frequency_unit': 'X = ไม่มีการปรับเปลี่ยนหลังจากปี 2022', 'update_frequency_interval': '1', 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป', 'data_source': 'Baseball Refernce', 'data_format': '99 = Parquet', 'data_category': '1 = ข้อมูลสาธารณะ', 'license_id': '1=Creative Commons Attributions'}
Data Dictionary: {'Rk': 'Rank', 'Name': 'Player name', 'Age': "Player's age", 'Tm': 'Team', 'Lg': 'League', 'G': 'Games played', 'PA': 'Plate appearances', 'AB': 'At bats', 'R': 'Runs scored/allowed', 'H': 'Hits/hits allowed', '2B':

In [68]:
#Write to GCS
catalog_path = 'gcs://dsi310_bucket/group02/2023-12-14/'
with fs.open(path=catalog_path+'2022_MLB_Batting.parquet',mode='wb') as f: 
    pq.write_table(table3, f)
    
tree(fs,catalog_path)


└── 2022_MLB_Batting.parquet


## Upload time

In [56]:
from google.cloud import storage
from datetime import datetime

def get_upload_timestamp(bucket_name, blob_name, token_path):
    """Retrieves the upload timestamp of a file in a GCS bucket."""
    # Authenticate using the token file
    storage_client = storage.Client.from_service_account_json(token_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    if blob:
        # Convert the timestamp to a readable format (ISO 8601)
        upload_timestamp = blob.time_created.isoformat()
        print(f"File {blob_name} was uploaded at {upload_timestamp}")
    else:
        print(f"File {blob_name} not found in bucket {bucket_name}.")

# Replace these with your actual values
gcs_token_path = 'teacher-dsi310-2023.json'  # Path to your service account JSON file
bucket_name = 'dsi310_bucket'  # Your GCS bucket name
blob_name = 'group02/2023-12-14/2022_MLB_Batting.parquet'  # Name of the file in GCS

get_upload_timestamp(bucket_name, blob_name, gcs_token_path)


File group02/2023-12-14/2022_MLB_Batting.parquet was uploaded at 2023-12-18T18:19:56.031000+00:00


## Data set2

In [75]:
MLB_Pitching_stats_df = pd.read_csv(r'C:\Users\ernie\Desktop\314 file\2022_MLB\Pitching.csv',encoding='latin-1')


#Metadata
metadata = {
 'data_type': '2 = ข้อมูลสถิติ',
 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)ในปี 2022',
 'owner_org': 'Sports-Reference',
 'maintainer': ' Baseball-Reference',
 'maintainer_email': 'sf@sports-reference.com',
 'tag_string': 'Rk,Name,W,L',
 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น',
 'update_frequency_unit': 'X = ไม่มีการปรับเปลี่ยนหลังจากปี 2022',
 'update_frequency_interval': '1',
 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป',
 'data_source':'Baseball Refernce',
 'data_format':'99 = Parquet',
 'data_category':'1 = ข้อมูลสาธารณะ',
 'license_id':'1=Creative Commons Attributions'
}
data_dictionary = {
    'Rk': 'Rank',
    'Name': 'Player name',
    'Age': "Player's age",
    'Tm': 'Team',
    'Lg': 'League',
    'W': 'Wins',
    'L': 'Losses',
    'W-L%': 'Win-Loss percentage',
    'ERA': '9 * ER / IP',
    'G': 'Games played',
    'GS': 'Games started',
    'GF': 'Games finished',
    'CG': 'Complete game',
    'SHO': 'Shutouts',
    'SV': 'Saves',
    'IP': 'Innings pitched',
    'H': 'Hits/Hits allowed',
    'R': 'Runs scored/allowed',
    'ER': 'Earned runs allowed',
    'HR': 'Home runs hit/allowed',
    'BB': 'Bases on balls/walks',
    'IBB': 'Intentional bases on balls',
    'SO': 'Strikeouts',
    'HBP': 'Times hit by a pitch',
    'BK': 'Balks',
    'WP': 'Wild pitches',
    'BF': 'Batters faced',
    'ERA+': '100 * (logERA/ERA)',
    'FIP': 'Fielding independent pitching. Measures a pitcher\'s effectiveness at HR, BB, HBP and causing SO.',
    'WHIP': '(BB + H) / IP',
    'H9': '9 * H / IP',
    'HR9': '9 * HR / IP',
    'BB9': '9 * BB / IP',
    'SO9': '9 * SO / IP',
    'SO/W': 'SO / W'
}



# Convert DataFrame to PyArrow Table with metadata
table4 = pa.Table.from_pandas(MLB_Pitching_stats_df)
table4 = table4.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table4.schema

Rk;Name;Age;Tm;Lg;W;L;W-L%;ERA;G;GS;GF;CG;SHO;SV;IP;H;R;ER;HR;BB;IBB;SO;HBP;BK;WP;BF;ERA+;FIP;WHIP;H9;HR9;BB9;SO9;SO/W: string
-- schema metadata --
metadata: '{'data_type': '2 = ข้อมูลสถิติ', 'title'' + 1259
dictionary: '{'Rk': 'Rank', 'Name': 'Player name', 'Age': "Player's age",' + 819

In [76]:
# Extracting metadata and data dictionary
metadata = eval(table4.schema.metadata[b'metadata']) if b'metadata' in table4.schema.metadata else None
data_dictionary = eval(table4.schema.metadata[b'dictionary']) if b'dictionary' in table4.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': '2 = ข้อมูลสถิติ', 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)ในปี 2022', 'owner_org': 'Sports-Reference', 'maintainer': ' Baseball-Reference', 'maintainer_email': 'sf@sports-reference.com', 'tag_string': 'Rk,Name,W,L', 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น', 'update_frequency_unit': 'X = ไม่มีการปรับเปลี่ยนหลังจากปี 2022', 'update_frequency_interval': '1', 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป', 'data_source': 'Baseball Refernce', 'data_format': '99 = Parquet', 'data_category': '1 = ข้อมูลสาธารณะ', 'license_id': '1=Creative Commons Attributions'}
Data Dictionary: {'Rk': 'Rank', 'Name': 'Player name', 'Age': "Player's age", 'Tm': 'Team', 'Lg': 'League', 'W': 'Wins', 'L': 'Losses', 'W-L%': 'Win-Loss percentage', 'ERA': '9 * ER / IP', 'G': 'Games played', 'GS': 'Games started', '

In [82]:
#Write to GCS
catalog_path = 'gcs://dsi310_bucket/group02/2023-12-14/'
with fs.open(path=catalog_path+'2022_MLB_Pitching.parquet',mode='wb') as f: 
    pq.write_table(table4, f)
    
tree(fs,catalog_path)

├── 2022_MLB_Batting.parquet
├── 2022_MLB_Pitching.parquet
└── Prospect.parquet


## Upload time

In [57]:
from google.cloud import storage
from datetime import datetime

def get_upload_timestamp(bucket_name, blob_name, token_path):
    """Retrieves the upload timestamp of a file in a GCS bucket."""
    # Authenticate using the token file
    storage_client = storage.Client.from_service_account_json(token_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    if blob:
        # Convert the timestamp to a readable format (ISO 8601)
        upload_timestamp = blob.time_created.isoformat()
        print(f"File {blob_name} was uploaded at {upload_timestamp}")
    else:
        print(f"File {blob_name} not found in bucket {bucket_name}.")

# Replace these with your actual values
gcs_token_path = 'teacher-dsi310-2023.json'  # Path to your service account JSON file
bucket_name = 'dsi310_bucket'  # Your GCS bucket name
blob_name = 'group02/2023-12-14/2022_MLB_Pitching.parquet'  # Name of the file in GCS

get_upload_timestamp(bucket_name, blob_name, gcs_token_path)


File group02/2023-12-14/2022_MLB_Pitching.parquet was uploaded at 2023-12-18T18:55:42.621000+00:00


## Data set3

In [78]:
Prospect_stats_df = pd.read_csv(r'C:\Users\ernie\Desktop\314 file\prospects.csv',encoding='latin-1')


#Metadata
metadata = {
 'data_type': '2 = ข้อมูลสถิติ',
 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'owner_org': 'Major League Baseball (MLB)',
 'maintainer': ' Major League Baseball Advanced Media (MLBAM)',
 'maintainer_email': 'dataprivacy@mlb.com',
 'tag_string': 'Top Prospect Batters',
 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น',
 'update_frequency_unit': 'A = ปี',
 'update_frequency_interval': '1',
 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป',
 'data_source':'Major League Baseball (MLB)',
 'data_format':'99 = Parquet',
 'data_category':'1 = ข้อมูลสาธารณะ',
 'license_id':'98 = License not specified'
}
data_dictionary = {
    'Rk': 'Rank of the player among prospects',
    'Name': 'Name of the player',
    'Age': 'Age of the player',
    'Tm': 'Team the player belongs to',
    'Lg': 'League in which the team plays',
    'PA': 'Plate Appearances',
    'AB': 'At Bats',
    'R': 'Runs scored',
    'H': 'Hits',
    '2B': 'Doubles',
    '3B': 'Triples',
    'HR': 'Home Runs',
    'TB': 'Total Bases',
    'RBI': 'Runs Batted In',
    'BB': 'Base on Balls',
    'SO': 'Strikeouts',
    'SB': 'Stolen Bases',
    'CS': 'Caught Stealing',
    'LOB': 'Left On Base',
    'HR%': 'Home Run Percentage',
    'BB%': 'Walk Percentage',
    'K%': 'Strikeout Percentage',
    'AVG': 'Batting Average',
    'OBP': 'On-base Percentage',
    'SLG': 'Slugging Percentage',
    'OPS': 'On-base Plus Slugging'
}



# Convert DataFrame to PyArrow Table with metadata
table5 = pa.Table.from_pandas(Prospect_stats_df)
table5 = table5.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table5.schema

rank: int64
team: string
age: int64
teamId: double
full_name: string
playerId: int64
atBats: int64
runs: int64
hits: int64
doubles: int64
triples: int64
homeRuns: int64
obp: double
ops: double
slg: double
rbi: int64
baseOnBalls: int64
strikeOuts: int64
stolenBases: int64
caughtStealing: int64
leftOnBase: int64
totalBases: int64
avg: double
position: string
sportAbbrev: string
-- schema metadata --
metadata: '{'data_type': '2 = ข้อมูลสถิติ', 'title'' + 1206
dictionary: '{'Rk': 'Rank of the player among prospects', 'Name': 'Name o' + 610

In [79]:
# Extracting metadata and data dictionary
metadata = eval(table5.schema.metadata[b'metadata']) if b'metadata' in table5.schema.metadata else None
data_dictionary = eval(table5.schema.metadata[b'dictionary']) if b'dictionary' in table5.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': '2 = ข้อมูลสถิติ', 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'owner_org': 'Major League Baseball (MLB)', 'maintainer': ' Major League Baseball Advanced Media (MLBAM)', 'maintainer_email': 'dataprivacy@mlb.com', 'tag_string': 'Top Prospect Batters', 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น', 'update_frequency_unit': 'A = ปี', 'update_frequency_interval': '1', 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป', 'data_source': 'Major League Baseball (MLB)', 'data_format': '99 = Parquet', 'data_category': '1 = ข้อมูลสาธารณะ', 'license_id': '98 = License not specified'}
Data Dictionary: {'Rk': 'Rank of the player among prospects', 'Name': 'Name of the player', 'Age': 'Age of the player', 'Tm': 'Team the player belongs to', 'Lg': 'League in which the team plays', 'PA': 'Plate Appearances', 'AB'

In [81]:
#Write to GCS
catalog_path = 'gcs://dsi310_bucket/group02/2023-12-14/'
with fs.open(path=catalog_path+'Prospect.parquet',mode='wb') as f: 
    pq.write_table(table5, f)
    
tree(fs,catalog_path)

├── 2022_MLB_Batting.parquet
├── 2022_MLB_Pitching.parquet
└── Prospect.parquet


## Upload time

In [58]:
from google.cloud import storage
from datetime import datetime

def get_upload_timestamp(bucket_name, blob_name, token_path):
    """Retrieves the upload timestamp of a file in a GCS bucket."""
    # Authenticate using the token file
    storage_client = storage.Client.from_service_account_json(token_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    if blob:
        # Convert the timestamp to a readable format (ISO 8601)
        upload_timestamp = blob.time_created.isoformat()
        print(f"File {blob_name} was uploaded at {upload_timestamp}")
    else:
        print(f"File {blob_name} not found in bucket {bucket_name}.")

# Replace these with your actual values
gcs_token_path = 'teacher-dsi310-2023.json'  # Path to your service account JSON file
bucket_name = 'dsi310_bucket'  # Your GCS bucket name
blob_name = 'group02/2023-12-14/Prospect.parquet'  # Name of the file in GCS

get_upload_timestamp(bucket_name, blob_name, gcs_token_path)


File group02/2023-12-14/Prospect.parquet was uploaded at 2023-12-18T18:55:29.699000+00:00


## Data set4

In [83]:
Batter_stats_df = pd.read_excel(r'C:\Users\ernie\Desktop\314 file\main pj\batter stats.xlsx')


#Metadata
metadata = {
 'data_type': '2 = ข้อมูลสถิติ',
 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'owner_org': 'Major League Baseball (MLB)',
 'maintainer': ' Major League Baseball Advanced Media (MLBAM)',
 'maintainer_email': 'mlbnviewerfeedback@mlb.com',
 'tag_string': 'Batters',
 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น',
 'update_frequency_unit': 'A = ปี',
 'update_frequency_interval': '1',
 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป',
 'data_source':'Major League Baseball (MLB)',
 'data_format':'99 = Parquet',
 'data_category':'1 = ข้อมูลสาธารณะ',
 'license_id':'98 = License not specified'
}
data_dictionary = {
    'Rk.': 'Rank',
    'Player': 'Player Name',
    'Pitches': 'Total Number of Pitches Faced',
    'Total': 'Total Number of (Specific Statistic)',
    'Pitch %': 'Percentage of Total Pitches',
    'PA': 'Plate Appearances',
    'BIP': 'Balls in Play',
    'Hits': 'Number of Hits',
    '3B': 'Triples',
    'HR': 'Home Runs',
    'K': 'Strikeouts',
    'BB': 'Base on Balls/Walks',
    'HBP': 'Hit By Pitch',
    'SF': 'Sacrifice Flies',
    'SH': 'Sacrifice Hits/Bunts',
    'XBH': 'Extra-Base Hits',
    'GB/FB': 'Ground Ball/Fly Ball Ratio',
    'LD%': 'Line Drive Percentage',
    'GB%': 'Ground Ball Percentage',
    'FB%': 'Fly Ball Percentage',
    'IFFB%': 'Infield Fly Ball Percentage',
    'HR/FB': 'Home Runs per Fly Ball Ratio',
    'O-Swing%': 'Swing Percentage at Pitches Outside the Strike Zone',
    'Z-Swing%': 'Swing Percentage at Pitches Inside the Strike Zone',
    'Swing%': 'Overall Swing Percentage',
    'O-Contact%': 'Contact Rate on Pitches Outside the Strike Zone',
    'Z-Contact%': 'Contact Rate on Pitches Inside the Strike Zone',
    'Contact%': 'Overall Contact Rate',
    'Zone%': 'Percentage of Pitches in the Strike Zone',
    'F-Strike%': 'First-Pitch Strike Percentage',
    'SwStr%': 'Swinging Strike Percentage',
    'Downward Movement w/ Gravity (in)': 'Downward Movement of the Pitch with Gravity in Inches',
    'Glove/Arm-Side Movement (in)': 'Lateral Movement of the Pitch Toward the Glove/Arm Side in Inches',
    'Vertical Movement w/o Gravity (in)': 'Vertical Movement of the Pitch without Gravity in Inches',
    'Movement Toward/Away from Batter (in)': 'Horizontal Movement of the Pitch Toward or Away from the Batter in Inches',
    'EV (MPH)': 'Exit Velocity in Miles Per Hour',
    'LA (°)': 'Launch Angle in Degrees',
    'Dist (ft)': 'Average Distance of Batted Balls in Feet',
    'Hard Hit%': 'Percentage of Hard Hit Balls',
    'Barrel/BBE%': 'Percentage of Barreled Balls per Balls in Play',
    'Barrel/PA%': 'Percentage of Barreled Balls per Plate Appearance'
}

# Convert DataFrame to PyArrow Table with metadata
table6 = pa.Table.from_pandas(Batter_stats_df)
table6 = table6.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table6.schema

Rk.: int64
Player: string
Pitches: int64
Total: int64
Pitch %: double
PA: int64
BIP: int64
Hits: int64
3B: int64
HR: int64
SO: int64
K%: double
BB: int64
BB%: double
BA: double
xBA: double
OBP: double
xOBP: double
SLG: double
xSLG: double
wOBA: double
xwOBA: double
Barrels: int64
ISO: double
Batter Run Value: double
Pitcher Run Value: double
Pitch (MPH): double
Spin (RPM): int64
Downward Movement w/ Gravity (in): double
Glove/Arm-Side Movement (in): double
Vertical Movement w/o Gravity (in): double
Movement Toward/Away from Batter (in): double
EV (MPH): double
LA (°): double
Dist (ft): int64
Hard Hit%: double
Barrel/BBE%: double
Barrel/PA%: double
-- schema metadata --
metadata: '{'data_type': '2 = ข้อมูลสถิติ', 'title'' + 1200
dictionary: '{'Rk.': 'Rank', 'Player': 'Player Name', 'Pitches': 'Total N' + 1800

In [84]:
# Extracting metadata and data dictionary
metadata = eval(table6.schema.metadata[b'metadata']) if b'metadata' in table6.schema.metadata else None
data_dictionary = eval(table6.schema.metadata[b'dictionary']) if b'dictionary' in table6.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': '2 = ข้อมูลสถิติ', 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'owner_org': 'Major League Baseball (MLB)', 'maintainer': ' Major League Baseball Advanced Media (MLBAM)', 'maintainer_email': 'mlbnviewerfeedback@mlb.com', 'tag_string': 'Batters', 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น', 'update_frequency_unit': 'A = ปี', 'update_frequency_interval': '1', 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป', 'data_source': 'Major League Baseball (MLB)', 'data_format': '99 = Parquet', 'data_category': '1 = ข้อมูลสาธารณะ', 'license_id': '98 = License not specified'}
Data Dictionary: {'Rk.': 'Rank', 'Player': 'Player Name', 'Pitches': 'Total Number of Pitches Faced', 'Total': 'Total Number of (Specific Statistic)', 'Pitch %': 'Percentage of Total Pitches', 'PA': 'Plate Appearances', 'BIP': 'Balls

In [85]:
#Write to GCS
catalog_path = 'gcs://dsi310_bucket/group02/2023-12-14/'
with fs.open(path=catalog_path+'Batter_stats.parquet',mode='wb') as f: 
    pq.write_table(table6, f)
    
tree(fs,catalog_path)

├── 2022_MLB_Batting.parquet
├── 2022_MLB_Pitching.parquet
├── Batter_stats.parquet
└── Prospect.parquet


## Upload time

In [60]:
from google.cloud import storage
from datetime import datetime

def get_upload_timestamp(bucket_name, blob_name, token_path):
    """Retrieves the upload timestamp of a file in a GCS bucket."""
    # Authenticate using the token file
    storage_client = storage.Client.from_service_account_json(token_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    if blob:
        # Convert the timestamp to a readable format (ISO 8601)
        upload_timestamp = blob.time_created.isoformat()
        print(f"File {blob_name} was uploaded at {upload_timestamp}")
    else:
        print(f"File {blob_name} not found in bucket {bucket_name}.")

# Replace these with your actual values
gcs_token_path = 'teacher-dsi310-2023.json'  # Path to your service account JSON file
bucket_name = 'dsi310_bucket'  # Your GCS bucket name
blob_name = 'group02/2023-12-14/Batter_stats.parquet'  # Name of the file in GCS

get_upload_timestamp(bucket_name, blob_name, gcs_token_path)


File group02/2023-12-14/Batter_stats.parquet was uploaded at 2023-12-18T19:02:34.216000+00:00


## Data set5

In [86]:
Pitcher_stats_df = pd.read_excel(r'C:\Users\ernie\Desktop\314 file\main pj\pitcher stats.xlsx')


#Metadata
metadata = {
 'data_type': '2 = ข้อมูลสถิติ',
 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'owner_org': 'Major League Baseball (MLB)',
 'maintainer': ' Major League Baseball Advanced Media (MLBAM)',
 'maintainer_email': 'mlbnviewerfeedback@mlb.com',
 'tag_string': 'Batters',
 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)',
 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น',
 'update_frequency_unit': 'A = ปี',
 'update_frequency_interval': '1',
 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป',
 'data_source':'Major League Baseball (MLB)',
 'data_format':'99 = Parquet',
 'data_category':'1 = ข้อมูลสาธารณะ',
 'license_id':'98 = License not specified'
}
data_dictionary_pitcher = {
    'Rk.': 'Rank',
    'Player': 'Player Name',
    'Pitches': 'Total Number of Pitches Thrown',
    'Total': 'Total Number of (Specific Statistic)',
    'Pitch %': 'Percentage of Total Pitches',
    'PA': 'Plate Appearances Against',
    'BIP': 'Balls in Play Against',
    'Hits': 'Number of Hits Allowed',
    '3B': 'Triples Allowed',
    'HR': 'Home Runs Allowed',
    'K': 'Strikeouts',
    'BB': 'Base on Balls/Walks Allowed',
    'HBP': 'Hit By Pitch',
    'SF': 'Sacrifice Flies Allowed',
    'SH': 'Sacrifice Hits/Bunts Allowed',
    'XBH': 'Extra-Base Hits Allowed',
    'GB/FB': 'Ground Ball/Fly Ball Ratio',
    'LD%': 'Line Drive Percentage Allowed',
    'GB%': 'Ground Ball Percentage Allowed',
    'FB%': 'Fly Ball Percentage Allowed',
    'IFFB%': 'Infield Fly Ball Percentage',
    'HR/FB': 'Home Runs per Fly Ball Ratio',
    'O-Swing%': 'Swing Percentage at Pitches Outside the Strike Zone',
    'Z-Swing%': 'Swing Percentage at Pitches Inside the Strike Zone',
    'Swing%': 'Overall Swing Percentage',
    'O-Contact%': 'Contact Rate on Pitches Outside the Strike Zone',
    'Z-Contact%': 'Contact Rate on Pitches Inside the Strike Zone',
    'Contact%': 'Overall Contact Rate',
    'Zone%': 'Percentage of Pitches in the Strike Zone',
    'F-Strike%': 'First-Pitch Strike Percentage',
    'SwStr%': 'Swinging Strike Percentage',
    'Downward Movement w/ Gravity (in)': 'Downward Movement of the Pitch with Gravity in Inches',
    'Glove/Arm-Side Movement (in)': 'Lateral Movement of the Pitch Toward the Glove/Arm Side in Inches',
    'Vertical Movement w/o Gravity (in)': 'Vertical Movement of the Pitch without Gravity in Inches',
    'Movement Toward/Away from Batter (in)': 'Horizontal Movement of the Pitch Toward or Away from the Batter in Inches',
    'EV (MPH)': 'Exit Velocity in Miles Per Hour',
    'LA (°)': 'Launch Angle in Degrees',
    'Dist (ft)': 'Average Distance of Batted Balls in Feet',
    'Hard Hit%': 'Percentage of Hard Hit Balls',
    'Barrel/BBE%': 'Percentage of Barreled Balls per Balls in Play',
    'Barrel/PA%': 'Percentage of Barreled Balls per Plate Appearance'
}


# Convert DataFrame to PyArrow Table with metadata
table7 = pa.Table.from_pandas(Pitcher_stats_df)
table7 = table7.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table7.schema

Rk.: int64
Player: string
Pitches: int64
Total: int64
Pitch %: double
PA: int64
BIP: int64
Hits: int64
3B: int64
HR: int64
SO: int64
K%: double
BB: int64
BB%: double
BA: double
xBA: double
OBP: double
xOBP: double
SLG: double
xSLG: double
wOBA: double
xwOBA: double
Barrels: int64
ISO: double
Batter Run Value: double
Pitcher Run Value: double
Pitch (MPH): double
Spin (RPM): int64
Downward Movement w/ Gravity (in): double
Glove/Arm-Side Movement (in): double
Vertical Movement w/o Gravity (in): double
Movement Toward/Away from Batter (in): double
EV (MPH): double
LA (°): double
Dist (ft): int64
Hard Hit%: double
Barrel/BBE%: double
Barrel/PA%: double
-- schema metadata --
metadata: '{'data_type': '2 = ข้อมูลสถิติ', 'title'' + 1200
dictionary: '{'Rk.': 'Rank', 'Player': 'Player Name', 'Pitches': 'Total N' + 1800

In [87]:
# Extracting metadata and data dictionary
metadata = eval(table7.schema.metadata[b'metadata']) if b'metadata' in table7.schema.metadata else None
data_dictionary = eval(table7.schema.metadata[b'dictionary']) if b'dictionary' in table7.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': '2 = ข้อมูลสถิติ', 'title': 'ชุดข้อมูลสถิติทางการเล่นเบสบอลในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'owner_org': 'Major League Baseball (MLB)', 'maintainer': ' Major League Baseball Advanced Media (MLBAM)', 'maintainer_email': 'mlbnviewerfeedback@mlb.com', 'tag_string': 'Batters', 'notes': 'ข้อมูลสถิติการเล่นเบสบอลของแต่ละผู้เล่นในระดับเมเจอร์ลีกเบสบอลอเมริกัน(MLB)', 'objective': '99 = อื่น ๆเพื่อวิเคราะห์ผู้เล่นเบสบอลในการหาปัจจัยสำคัญที่ทำให้มีโอกาสชนะมากขึ้น', 'update_frequency_unit': 'A = ปี', 'update_frequency_interval': '1', 'geo_coverage': '02 = ทวีป / กลุ่มประเทศในทวีป', 'data_source': 'Major League Baseball (MLB)', 'data_format': '99 = Parquet', 'data_category': '1 = ข้อมูลสาธารณะ', 'license_id': '98 = License not specified'}
Data Dictionary: {'Rk.': 'Rank', 'Player': 'Player Name', 'Pitches': 'Total Number of Pitches Faced', 'Total': 'Total Number of (Specific Statistic)', 'Pitch %': 'Percentage of Total Pitches', 'PA': 'Plate Appearances', 'BIP': 'Balls

In [88]:
#Write to GCS
catalog_path = 'gcs://dsi310_bucket/group02/2023-12-14/'
with fs.open(path=catalog_path+'Pitcher_stats.parquet',mode='wb') as f: 
    pq.write_table(table6, f)
    
tree(fs,catalog_path)

├── 2022_MLB_Batting.parquet
├── 2022_MLB_Pitching.parquet
├── Batter_stats.parquet
├── Pitcher_stats.parquet
└── Prospect.parquet


## Upload time

In [61]:
from google.cloud import storage
from datetime import datetime

def get_upload_timestamp(bucket_name, blob_name, token_path):
    """Retrieves the upload timestamp of a file in a GCS bucket."""
    # Authenticate using the token file
    storage_client = storage.Client.from_service_account_json(token_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    if blob:
        # Convert the timestamp to a readable format (ISO 8601)
        upload_timestamp = blob.time_created.isoformat()
        print(f"File {blob_name} was uploaded at {upload_timestamp}")
    else:
        print(f"File {blob_name} not found in bucket {bucket_name}.")

# Replace these with your actual values
gcs_token_path = 'teacher-dsi310-2023.json'  # Path to your service account JSON file
bucket_name = 'dsi310_bucket'  # Your GCS bucket name
blob_name = 'group02/2023-12-14/Pitcher_stats.parquet'  # Name of the file in GCS

get_upload_timestamp(bucket_name, blob_name, gcs_token_path)

File group02/2023-12-14/Pitcher_stats.parquet was uploaded at 2023-12-18T19:07:14.640000+00:00
